In [33]:
import arcpy
from arcpy.sa import Slope

# Set environment settings
arcpy.env.workspace = r"C:\LiDAR_2"
arcpy.env.overwriteOutput = True


# Input and output paths
input_laz = r"C:\LiDAR_2\2024\Lone_Oaks_2024-05-01_Raw.copc.laz" #takes the point cloud as an input as .laz
converted_las = r"C:\LiDAR_2\project_output\\" #to store the .las from .laz
las_dataset = r"C:\LiDAR_2\lasdataset\dataset.lasd" #to store .lasd dataset
output_raster = r"C:\LiDAR_2\raster\dem.tif" #to store DEM raster
slope_model = r"C:\LiDAR_2\raster\slope.tif" #to store slope model

# NAD 83 State Plane Tennessee FIPS 4100 (Feet)
spatial_reference = arcpy.SpatialReference(2274)  # EPSG Code for NAD 1983 Tennessee State Plane 

#convert LAZ to LAS
arcpy.conversion.ConvertLas(
    input_laz,
    converted_las,
    "1.4"# LAS file version
    "NO_COMPRESSION"
)

#create LAS Dataset
arcpy.management.CreateLasDataset(
    [r"C:\LiDAR_2\project_output\Lone_Oaks_2024-05-01_Raw.copc.las"],  #takes the las file
    las_dataset,
    "RECURSION", #to loop through all files in the folder
    None, #not sure why, but after many attempts this worked
    spatial_reference 
)


#classify Ground
arcpy.ddd.ClassifyLasGround(
    in_las_dataset = las_dataset,
    method = "CONSERVATIVE", #method to classify ground to non-ground points
    dem_resolution = 5, #resolution for sampling
    reuse_ground="RECLASSIFY_GROUND",
    dem_resolution="0.5 Meters",
    compute_stats="COMPUTE_STATS",
    extent="DEFAULT",
    boundary=None,
    process_entire_files="PROCESS_EXTENT",
    update_pyramid="UPDATE_PYRAMID",
    algorithm="LATEST",
    classify_low_noise="NO_CLASSIFY_LOW_NOISE",
    minimum_depth_below_ground="0.25 Meters",
    preserve_low_noise="RECLASSIFY_LOW_NOISE",
    classify_high_noise="NO_CLASSIFY_HIGH_NOISE",
    minimum_height_above_ground="100 Meters",
    preserve_high_noise="RECLASSIFY_HIGH_NOISE"
)


#export LAS Dataset to Raster
arcpy.conversion.LasDatasetToRaster(
    in_las_dataset=las_dataset,
    out_raster = output_raster,
    value_field="ELEVATION",  # Field to base raster values on
    interpolation_type="BINNING MAXIMUM NATURAL_NEIGHBOR", #method to interpolate the point cloud to raster
    data_type="FLOAT", #type for the cell
    sampling_type="CELLSIZE",
    sampling_value=.5  #cell size 
)


print("Process completed successfully")



'\n# Step 5: Create Slope Model\narcpy.CheckOutExtension("Spatial")  # Check out Spatial Analyst extension\nslope = Slope(output_raster, output_measurement="DEGREE")  # Calculate slope\nslope.save(slope_model)  # Save the slope model\narcpy.CheckInExtension("Spatial")  # Check in Spatial Analyst extension\n\nprint("Process completed successfully!")\n'